In [1]:
from sources.interface_selecao_cortes_3_0 import *
import warnings
import getpass 


warnings.filterwarnings('ignore') 
 
#usuario = "ETB348056"
#Usuário e Senha Hana

usuario = input('Usuário: ')
senha = getpass.getpass('Senha: ')

Usuário: ETB348056
Senha: ········


In [2]:
atena = SelecaoLayout(usuario, senha)

In [3]:
atena.show()

In [ ]:
atena.salvar_maiores_devedores_excel(b)

In [ ]:
atena.df_maiores_devedores

In [ ]:
'/'.join([str(locale.format("%.2f",atena.selecionados_sm.loc[atena.selecionados_sm['cluster_id']==i]['PECLD'].sum(),grouping = True)) for i in atena.selecionados_sm.cluster_id.unique()])

In [ ]:
atena.selecionados_sm

In [ ]:
atena.html_centroides.layout.height = '500px'

In [ ]:
if (atena.carteira_def.value):
    print('yo')

In [ ]:
','.join([str(int(a.loc[a['cluster_id']==i]['dist'].max())) for i in a.cluster_id.unique()])
 

In [ ]:
t = [i for i in atena.selecao.selecionados.cluster.unique()]

In [ ]:
a = atena.selecao.selecionados.copy()

In [ ]:
import locale 
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

In [ ]:
atena.selecao.dfn['MTV_COB'] = locale.format("%.2f",atena.selecao.dfn['MTV_COB'],grouping = True)

In [ ]:
atena.importar_clusterconf(atena.botao_importar_conf)

In [ ]:


sql = """SELECT MARCACAO,HORA FROM {tabela} 
                WHERE DATA_PEDIDO = CURRENT_DATE 
                AND MARCACAO IS NOT NULL
                AND HORA IS NOT NULL
                GROUP BY MARCACAO, HORA """.format(tabela='CLB961920.ATENA_HIST')

try:
    connection_hana = dbapi.connect(
                   address='BRNEO695',
                   port='30015',
                   user=atena._SelecaoLayout__usuario,
                   password=atena._SelecaoLayout__senha,
                   databasename='BNP',
                   sslValidateCertificate=False
    )

    marc_grav = pd.read_sql(sql, connection_hana)

    if marc_grav.shape[0]>0:

        marc_grav = marc_grav.loc[marc_grav['HORA']==marc_grav['HORA'].max()]

        marc_grav = marc_grav['MARCACAO'].str[-3:]
        marc_grav = marc_grav.astype(int)
        marc_grav_max = marc_grav.max()
        #cluster na marcação máximo de 3 linhas
        if marc_grav_max == 999: marc_grav_max = 0
        atena.marc_insert = marc_grav_max
    else:
        atena.marc_insert = 0
except Exception as e:
    print(e)

In [ ]:
atena.marc_insert

In [ ]:
sql_conf

In [ ]:
atena.update_marcacao('resultados/STC_MISTA_1605.xlsx')

In [ ]:
 arquivo_hist = pd.read_excel('resultados/STC_MISTA_1605.xlsx',converters={'ZCGACCOUN': str,'ZONA': str, 'MARCACAO': str,'MARC_ZONA': str})

In [ ]:
def update_marcacao(arquivo,self):
 
    for i in arquivo.MARCACAO.unique():
        if arquivo.loc[arquivo['MARCACAO']==i]['MARC_ZONA'].unique()[:3][0] == 'MIX':
            if arquivo.loc[arquivo['MARCACAO'] == i].ZONA.unique().shape[0] == 1:
                arquivo.loc[arquivo['MARCACAO']==i,'MARC_ZONA'] = arquivo.loc[arquivo['MARCACAO'] == i].ZONA.unique()[0].lstrip('0')
        self.marc_insert = self.marc_insert + 1
        arquivo.loc[arquivo['MARCACAO']==i,'MARCACAO'] = str(today.day).zfill(2) + arquivo.loc[arquivo['MARCACAO']==i]['MARC_ZONA'].str.lstrip("0").str.zfill(3) + str(self.marc_insert).zfill(3)

    return arquivo

In [ ]:
t = update_marcacao(arquivo_hist,self=atena)

In [ ]:
t.MARCACAO.unique()

In [ ]:
atena.marc_insert

In [ ]:
connection_hana = dbapi.connect(
               address='BRNEO695',
               port='30015',
               user=atena._SelecaoLayout__usuario,
               password=atena._SelecaoLayout__senha,
               databasename='BNP',
               sslValidateCertificate=False
)

In [ ]:
sql_importar = """SELECT TURMA,ZCGUTD AS UTD,CARTEIRA,AVG(LATITUDE) AS LATITUDE, AVG(LONGITUDE)
AS LONGITUDE, MARCACAO, CAST("cluster" AS INT) AS "cluster",CAST(RAIO AS INT) AS RAIO, ZCGIDZONA AS ZONA

FROM CLB961920.ATENA_HIST ah 

WHERE DATA_PEDIDO = {data_pedido}
AND FLAG_DEF = 'N'

GROUP BY TURMA, CARTEIRA,ZCGUTD,MARCACAO, RAIO, ZCGIDZONA,"cluster" """.format(data_pedido = 'ADD_DAYS(CURRENT_DATE,-1)')

In [ ]:
conf = pd.read_sql(sql_importar, connection_hana)

In [ ]:
conf

In [ ]:
conf.loc[conf['ZONA']== 351]

In [ ]:
def get_saved_clusters():
        sql_importar = """SELECT TURMA,ZCGUTD AS UTD,CARTEIRA,AVG(LATITUDE) AS LATITUDE, AVG(LONGITUDE)
        AS LONGITUDE, MARCACAO, CAST("cluster" AS INT) AS "cluster",CAST(RAIO AS INT) AS RAIO, ZCGIDZONA AS ZONA,
        count(*) AS QTD, SUM(ZCGMTVCOB) AS MTV_COB,SUM(ZCGPECLD) AS PECLD

        FROM CLB961920.ATENA_HIST ah 

        WHERE DATA_PEDIDO = {data_pedido}
        AND FLAG_DEF = 'N'

        GROUP BY TURMA, CARTEIRA,ZCGUTD,MARCACAO, RAIO, ZCGIDZONA,"cluster" """.format(data_pedido = 'ADD_DAYS(CURRENT_DATE,-1)') 
        
        connection_hana = dbapi.connect(
               address='BRNEO695',
               port='30015',
               user=atena._SelecaoLayout__usuario,
               password=atena._SelecaoLayout__senha,
               databasename='BNP',
               sslValidateCertificate=False)
        
        
        
        clusters = pd.read_sql(sql_importar, connection_hana)
        
        return clusters
        
        

In [ ]:
def preencher_cluster(self,marcacao,UTD: str=None, locali: list = None,zona: list = None, 
                municipio: list = None, bairros: list = None, local: str = None,
                preencher: str = None, qtd_preencher:int = 0):

    """
    Função que realiza o preenchimento de um cluster com outro tipo de serviço

    :param UTD: UTD para filtrar os clientes
    :param zona: Zona para filtrar os clientes
    :param bairros: bairros para filtrar os clientes
    :param municipio: municipio dos clientes a serem selecionados
    :param r_max_preciso: maior precisão na seleção do raio máximo do cluster, mas pode diminui o IRR
    :param min_selecionados: número mínimos de clientes a serem selecionados por cluster original
    :preencher: carteira para realizar o preenchimento do cluster
    :qtd_preencher: quantidade de clientes para ser adicionado ao cluster, caso seja passado o valor 0 o cluster original será preenchido conforme defasagem
    """             


    selecionados_tot = pd.DataFrame()
    accoun_selecionadas = []

    ucs = self.selecao.dfn.copy()
    ucs = ucs.dropna(subset=['LATITUDE', 'LONGITUDE'])

    centroides = get_saved_clusters()
    centroides = centroides.loc[centroides['MARCACAO'].isin(marcacao)].reset_index()

    
    if preencher and 'SERVICO' in ucs:
        ucs = ucs.loc[ucs['SERVICO'] == preencher]
    
    elif 'SERVICO' in ucs:
        ucs = ucs.loc[ucs['SERVICO']==centroides['CARTEIRA'].iloc[0]]
        
    
    
    if UTD:
        ucs = ucs.loc[ucs['UTD'] == UTD]

    if zona:
        
        for i in range(len(zona)):
            zona[i] = zona[i].zfill(4)
            
        ucs = ucs.loc[ucs['ZONA'].isin(zona)]
    if municipio:
        ucs = ucs.loc[ucs['ZCGMUNICI'].isin(municipio)]

    if bairros:
        ucs = ucs.loc[ucs['ZCGBAIRRO'].isin(bairros)]
    if locali:
        ucs = ucs.loc[ucs['ZCGLOCALI'].isin(locali)]
    if local == 'R':
        ucs = ucs.loc[ucs['ZCGTIPLOC'] == 'R']
    elif local == 'U':
        ucs = ucs.loc[ucs['ZCGTIPLOC'] == 'U']
    

    if 'irr' not in ucs:
        ucs['irr'] = ucs['IRR']
        
    
            

    for i in range(centroides.MARCACAO.unique().shape[0]):
        ucs = ucs[~ucs.ZCGACCOUN.isin(accoun_selecionadas)]
        #centroide = centroides.loc[centroides['MARCACAO']==i]
        
        r = centroides['RAIO'].iloc[i]
     
        carteira = centroides['CARTEIRA'].iloc[i]
        
        if qtd_preencher > 0:

            ucs['raio'] = self.selecao.calcular_distancia(centroides.iloc[i][['LATITUDE', 'LONGITUDE']],
                                                            ucs[['LATITUDE', 'LONGITUDE']]) 
            
            selecionados = ucs.loc[ucs['raio'] <= r].sort_values('irr', ascending=False)[:qtd_preencher]


            selecionados['dist'] = self.selecao.calcular_distancia(
                centroides.iloc[i][['LATITUDE', 'LONGITUDE']],
                selecionados[['LATITUDE', 'LONGITUDE']])

           #5
            selecionados.drop(columns = ['raio'],inplace=True)
            selecionados['CARTEIRA'] = carteira
            selecionados['cluster'] =  centroides['cluster'].iloc[i]
            selecionados['MARCACAO'] = centroides['MARCACAO'].iloc[i]
            #selecionados['MARC_ZONA'] = self.selecionados.loc[self.selecionados['cluster']==i]['MARC_ZONA'].iloc[0]
            selecionados['RAIO'] = r
            selecionados['BYPASS'] = True

            accoun_selecionadas.append(selecionados['ZCGACCOUN'].to_list())

            selecionados_tot = selecionados_tot.append(selecionados)

   
    #self.selecionados = self.selecionados.append(sel ecionados_tot)
    

    return selecionados_tot

In [ ]:
preencher = preencher_cluster(self = atena,marcacao = ['18233043'],
                              qtd_preencher = 3)

In [ ]:
preencher

In [ ]:
preencher.loc[preencher['MARCACAO'].str[:2] == str(today.day-1)]

In [ ]:
preencher['CARTEIRA']

In [ ]:
clusters = get_saved_clusters()

In [ ]:
clusters.iloc[0][['LATITUDE','LONGITUDE']]

In [ ]:
clusters.loc[clusters['MARCACAO']=='18233043']

In [ ]:
t = ['10','20','30']

In [ ]:
for i in range(len(t)):
    t[i] = t[i].zfill(4)

In [ ]:
table="""<table>
                    <tr>
                        <th>TURMA </th>
                        <th>UTD </th>
                        <th>CARTEIRA</th>
                        <th>CLUSTER</th>
                        <th>RAIO (m)</th>
                        <th>MTVCOB (R$)</th>
                        <th>PECLD CONS (R$)</th>
                        <th>CLIENTES</th>
                    </tr>
                  """

In [ ]:
for i in range(clusters.shape[0]):
    table += """<tr>
                        <td>{0}</td>
                        <td>{1}</td>
                        <td>{2}</td>
                        <td>{3}</td>
                        <td>{4}</td>
                        <td>{5}</td>
                        <td>{6}</td>
                        <td>{7}</td>
                    </tr>
                  """.format(clusters['TURMA'].iloc[i],
                            clusters['UTD'].iloc[i],
                            clusters['CARTEIRA'].iloc[i],
                            clusters['cluster'].iloc[i],
                            clusters['RAIO'].iloc[i],
                            locale.format("%.2f",round(clusters['MTV_COB'].iloc[i],2),grouping = True),
                            locale.format("%.2f",round(clusters['PECLD'].iloc[i],2),grouping = True),
                            clusters['QTD'].iloc[i])

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
b=widgets.HTML(
    value= table,
    disabled=True
)
a = widgets.HBox([b], layout=Layout(height='500px', overflow_y='auto'))
display(a)

In [ ]:
import locale

In [ ]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

In [ ]:
locale.currency(round(clusters['MTV_COB'].iloc[i],2),grouping=True)

In [ ]:
locale.format("%.2f", round(clusters['MTV_COB'].iloc[i],2), grouping = True)

In [ ]:
table="""<table>
                    <tr>
                        <th>ZCGACCOUN </th>
                        <th>UTD </th>
                        <th>ZCGMUNICI</th>
                        <th>ZONA</th>
                        <th>LOCALIDADE</th>
                        <th>MTVCOB (R$)</th>
                        <th>PECLD CONS (R$)</th>
                        <th>ZCGQTFTVE</th>
                    </tr>
                  """

In [ ]:
atena.selecao.dfn['ZCGQTFTVE'].iloc[1]

In [ ]:
t = atena.selecao.dfn.loc[atena.selecao.dfn['UTD']=='GRACA'].head(5).reset_index()

In [ ]:
locale.format("%.2f",atena.selecao.dfn['PECLD'].max(),grouping=True)

In [ ]:
teste = ['a','b']

In [ ]:
teste + [""]